
# Elección de la mejor predición


VAmos a trabajar con las sigueinte  __Series de tiempos __ que nos da ventas mensuales de coles en Quebec desde 1960 hasta 1968. Este dataset puede ser bajado de aquí: 

<https://datamarket.com/data/set/22n4/monthly-car-sales-in-quebec-1960-1968>

El objetivo de este notebook es explorar diferentes modelos para predecir 
las ventas futuras. 

Seguiremos los siguientes pasos:

1. Preparación de datos.
2. Separar los datos en entrenamiento, test y validación.
3. Construir los diferentes modelos.
4. Ajustar los modelos o a los datos de entrenamiento.
5. Evaluar cada modelo en el conjunto de valildación y elegir el mejor.
6. Evaluar el modelo en el conjunto de test.

In [ ]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]
plt.rc('xtick', labelsize=20)     
plt.rc('ytick', labelsize=20)

### Preparación de datos

In [ ]:
cars_df = pd.read_csv("TUPATH")
cars_df.tail()

Limpiamos última fila

In [ ]:
cars_df = cars_df.iloc[:-1]

Cambiamos el formato de las fechas

In [ ]:
cars_df["dt"] = cars_df["Month"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m'))

Renombramos la columna `Monthly car sales in Quebec 1960-1968` por  `sales` y eliminamos la columna `Month`.

In [ ]:
cars_df = cars_df\
    .rename({"Monthly car sales in Quebec 1960-1968": "sales"}, axis=1)\
    .drop("Month", axis=1)
cars_df.tail()

Dibujamos nuestra serie de tiempos:

In [ ]:
plt.plot(cars_df["dt"], cars_df["sales"])

Verificamos que tenemos datos de todos los meses:

In [ ]:
cars_df[['dt']].groupby(cars_df["dt"].dt.year).count()

## Separar los datos en entrenamiento, test y validación.


* Mucho cuidado que cuando trabajamos series de tiempos no podemos hacer el split aleatoriamente ya que se trata de predecir datos a futuro. Por tanto tenemos respetar que el conjunto de validación y test son usando datos posteriores al conjunto de entrenamiento.

In [ ]:
#WRITE YOUR CODE

## Modelo 1: media de las anteriores ventas


Un simple modelo es predecir a futuro el valor de la media de los datos anteriores.

In [ ]:
#WRITE YOUR CODE

Dibujamos su performance

In [ ]:
def plot_predicition(X_train, y_train, X_dev, y_dev, y_dev_hat):
    plt.plot(X_train["dt"], y_train, c="blue", label='train data')
    plt.plot(X_dev["dt"], y_dev, c="green", label='dev data')
    plt.plot(X_dev["dt"], y_dev_hat, c="orange", label='prediction')
    plt.legend()
    plt.show()
    
plot_predicition(X_train, y_train, X_dev, y_dev, y_dev_hat)

Vemos que la predición no es buena, pero nos sirve para tener un __baseline__, un punto de comienzo para ir mejorando con otros modelos.

Para la evaluación del performance del modelo usaremos el error cuadrático medio, MSE, y su raiz, RSME que nos servirán para elegir el mejor modelo.

In [ ]:
from sklearn.metrics import mean_squared_error

print("MSE: " ,mean_squared_error(y_dev, y_dev_hat))
print("RMSE: ", np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

## Modelo 2: media del año anterior

Vamos a tomar la media de los últimos doce meses

In [ ]:
#WRITE YOUR CODE

This time it looks better.

In [ ]:
np.sqrt(mean_squared_error(y_dev, y_dev_hat))

##  Modelo 3: valor del mes anterior


A menudo, para predecir el valor del siguiente mes, usamos el valor del anterior mes. Para ello vamos a añadir una nueva feature que nos da el valor del mes anterior usando el método en python llamado `shift`. esto es llamado __lag__.

In [ ]:
#WRITE YOUR CODE

In [ ]:
X_train, X_dev, X_test, y_train, y_dev, y_test = split_into_train_dev_test(Xy_with_lags)

In [ ]:
y_dev_hat = X_dev['sales-1']
plot_predicition(X_train, y_train, X_dev, y_dev, y_dev_hat)

In [ ]:
np.sqrt(mean_squared_error(y_dev, y_dev_hat))

##  Modelo 4: modelo de Autoregresion (AR)


Es claro que los datos de Diciembre de un año y el anterior están correlacionados. Vamos a calcular si es cierto haciendo la función de correlación entre un periodo y el de un año anterior (usaremos lags).

In [ ]:
N_lags = #WRITE YOUR CODE

def create_lags(Xy, n_lags):
    Xy_with_lags = Xy.copy()
    for i in range(1, n_lags+1):
        Xy_with_lags['sales-'+str(i)] = Xy_with_lags['sales'].shift(i)
    return Xy_with_lags

Xy_with_lags = create_lags(Xy, N_lags)
autocorrelations = Xy_with_lags.iloc[N_lags:].drop("dt", axis=1).corr()['sales']

In [ ]:
autocorrelations.plot(kind='bar',color='gray')

Elegimos las columnas que son más correladas con sales.

In [ ]:
#WRITE YOUR CODE

Eliminamos los Nas de los lags y construimos un modelo lineal

In [ ]:
#WRITE YOUR CODE

In [ ]:
plot_predicition(X_train, y_train, X_dev, y_dev, y_dev_hat)

Ahora hemos mejorado más que el doble!!!!

##  Modelo 5: modelo recurrente 

A menudo en la práctica nosotros buscamos una predición para todo el año. Vamos a asumir que estamos en Enero de 1967 y nos gustaria tener una previsión par todos los meses de 1967 y así la empresa podría optimizar el stock mensual.

Podemos hacer lo mismo que en el modelo anterior pero usando las prediciones anteriores. Esto significa que para predecir Abril usaremos los ultimos 12 meses últimos, tomando para Enero, Febreo y Marzo los datos de predicion. 

In [ ]:
def recurrent_prediction(y_train, n_steps, reg):
    x_dev = list(y_train[-1:-(n_steps +1):-1])
    y_dev_hat = []
    for i in range(n_steps):
        y_hat = reg.predict([x_dev])[0]
        y_dev_hat.append(y_hat)
        x_dev.pop(-1)
        x_dev = [y_hat] + x_dev
    return y_dev_hat

y_dev_hat = recurrent_prediction(y_train, 12, reg_all_columns)
np.sqrt(mean_squared_error(y_dev, y_dev_hat))

In [ ]:
plot_predicition(X_train, y_train, X_dev, y_dev, y_dev_hat)

##  Modelo 6: K-nearest neighborhood 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

N_lags = #WRITE YOUR CODE
Xy_with_lags = create_lags(Xy, N_lags).iloc[N_lags:]
X_train, X_dev, X_test, y_train, y_dev, y_test = split_into_train_dev_test(Xy_with_lags)

reg_kn = KNeighborsRegressor(n_neighbors=#WRITE YOUR CODE)
reg_kn.fit(X_train.drop("dt", axis=1), y_train)
y_dev_hat = recurrent_prediction(y_train, N_lags, reg_kn)
np.sqrt(mean_squared_error(y_dev, y_dev_hat))

In [ ]:
plot_predicition(X_train, y_train, X_dev, y_dev, y_dev_hat)

 Elige tu mejor modelo para el parámetro
 n_neighbors

In [ ]:
#WRITE YOUR CODE

##  Modelo 7: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


reg_rf = RandomForestRegressor(random_state=667)
reg_rf.fit(X_train.drop("dt", axis=1), y_train)
y_dev_hat = recurrent_prediction(y_train, N_lags, reg_rf)
np.sqrt(mean_squared_error(y_dev, y_dev_hat))

Elige tu mejor modelo para los parámetros n_estimators y max_depth

In [ ]:
#WRITE YOUR CODE

In [ ]:
X_train_dev = pd.concat([X_train, X_dev])
y_train_dev = np.concatenate([y_train, y_dev])

reg_rf = RandomForestRegressor(n_estimators=#WRITE YOUR CODE,
                               max_depth=#WRITE YOUR CODE
                               , random_state=667)
reg_rf.fit(X_train_dev.drop("dt", axis=1), y_train_dev)
y_test_hat = recurrent_prediction(y_test, N_lags, reg_rf)
np.sqrt(mean_squared_error(y_test, y_test_hat))

In [ ]:
plot_predicition(X_train_dev, y_train_dev, X_test, y_test, y_test_hat)

## Modelo 8: Autoregression (AR) 

In [ ]:
from statsmodels.tsa.ar_model import AR
model = AR(y_train)
model_fit = model.fit(maxlag=12)
y_dev_hat = model_fit.predict(start=len(y_train), end=len(y_train)+11)    
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

In [ ]:
plot_predicition(X_train, y_train, X_dev, y_dev, y_dev_hat)

## Modelo 8: Autoregression Moving Average (ARIMA) 

In [ ]:
from statsmodels.tsa.arima_model import ARMA
model = ARMA(y_train, order=(9, 2))
model_fit = model.fit(disp=0, start_ar_lags=13)

In [ ]:
y_dev_hat = model_fit.predict(start=len(y_train), end=len(y_train)+11)    
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

In [ ]:
plot_predicition(X_train, y_train, X_dev, y_dev, y_dev_hat)

## Bibliography

https://machinelearningmastery.com/how-to-develop-a-skilful-time-series-forecasting-model/

https://www.itl.nist.gov/div898/handbook/pmc/section4/pmc41.htm

https://www.it.uu.se/research/publications/reports/2006-022/2006-022-nc.pdf
http://www.blackarbs.com/blog/time-series-analysis-in-python-linear-models-to-garch/11/1/2016
